In [1]:
from scipy.stats import spearmanr, rankdata
from scipy.spatial.distance import pdist, squareform

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import shutil
import time
import _pickle as cp
import sys
from astropy.stats import median_absolute_deviation
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler
import random
from sklearn.model_selection import train_test_split


from PIL import Image

In [2]:
basePath = '/content/drive/MyDrive/DATN/Code/Solution 2 - CIC-DDoS2019'

In [7]:
# đổi tên để sử dụng cho các tập data khác
dataset = 'CIC-DDoS2019'

# đường dẫn file csv
combind_csv_path = basePath + f'/datasets/{dataset}/csv/_combine_data.csv'

# chọn đặc trưng

In [4]:
# chọn đặc trưng

selected_columns = ['Total Fwd Packets', 'Total Backward Packets',
       'Fwd Packets Length Total', 'Bwd Packets Length Total',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Avg Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
       'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate',
       'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'Init Fwd Win Bytes', 'Init Bwd Win Bytes',
       'Fwd Act Data Packets', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

# Functions

In [5]:
# Hàm phân chia các ảnh thành 3 danh sách theo tỷ lệ 60%-20%-20%
def split_df(df):

    # 1. Lấy ngẫu nhiên 60% dữ liệu cho tập thứ nhất
    df_60 = df.sample(frac=0.6, random_state=42)

    # 2. Loại bỏ những dòng đã chọn từ DataFrame ban đầu
    df_remaining = df.drop(df_60.index)

    # 3. Lấy ngẫu nhiên 20% dữ liệu (trên phần dữ liệu còn lại) cho tập thứ hai
    df_20_1 = df_remaining.sample(frac=0.5, random_state=42)

    # 4. Phần còn lại chính là 20% cuối cùng cho tập thứ ba
    df_20_2 = df_remaining.drop(df_20_1.index)

    return df_60, df_20_1, df_20_2

In [6]:
# hàm chuyển df thành image với số lượng theo %

def convert_to_img_percent(data_normalized, label, split_1_directory, split_2_directory, split_3_directory):
    # Chuyển đổi mỗi hàng thành hình ảnh


    # Kiểm tra nếu thư mục đã tồn tại
    if not os.path.exists(f'{split_1_directory}/{label}'):
        os.makedirs(f'{split_1_directory}/{label}')

    if not os.path.exists(f'{split_2_directory}/{label}'):
        os.makedirs(f'{split_2_directory}/{label}')

    if not os.path.exists(f'{split_3_directory}/{label}'):
        os.makedirs(f'{split_3_directory}/{label}')

    split_60, split_20_1, split_20_2 = split_df(data_normalized)

    for index, row in split_60.iterrows():

        image_array = np.array(row).reshape((5, 15))  # hiện tại chọn 75 đặc trưng nên chọn reshape với kích thước 5x15 = 75
        image_array = np.nan_to_num(image_array)  # Thay thế các giá trị không hợp lệ bằng 0
        image = Image.fromarray((image_array * 255).astype(np.uint8)) # nhân 255 để chuyển sang giá trị RGB

        image = image.resize((224, 224))  # Chuyển đổi kích thước hình ảnh thành 224x224 (thường train model dùng kích thước này)
        image.save(f'{split_1_directory}/{label}/image_{index}.png')

    for index, row in split_20_1.iterrows():

        image_array = np.array(row).reshape((5, 15))  # hiện tại chọn 75 đặc trưng nên chọn reshape với kích thước 5x15 = 75
        image_array = np.nan_to_num(image_array)  # Thay thế các giá trị không hợp lệ bằng 0
        image = Image.fromarray((image_array * 255).astype(np.uint8)) # nhân 255 để chuyển sang giá trị RGB

        image = image.resize((224, 224))  # Chuyển đổi kích thước hình ảnh thành 224x224 (thường train model dùng kích thước này)
        image.save(f'{split_2_directory}/{label}/image_{index}.png')

    for index, row in split_20_2.iterrows():

        image_array = np.array(row).reshape((5, 15))  # hiện tại chọn 75 đặc trưng nên chọn reshape với kích thước 5x15 = 75
        image_array = np.nan_to_num(image_array)  # Thay thế các giá trị không hợp lệ bằng 0
        image = Image.fromarray((image_array * 255).astype(np.uint8)) # nhân 255 để chuyển sang giá trị RGB

        image = image.resize((224, 224))  # Chuyển đổi kích thước hình ảnh thành 224x224 (thường train model dùng kích thước này)
        image.save(f'{split_3_directory}/{label}/image_{index}.png')

# Đọc và hiển thị một hình ảnh mẫu
# image = Image.open('./Results/Test_3/LDAP-training/image_0.png')
# image.show()

## tách image train thành các tập: train, validate, test

In [8]:
# đọc file csv đã hợp nhất các file csv khác

# Đọc dữ liệu từ file CSV
full_data = pd.read_csv(combind_csv_path)

## Nhóm dữ liệu theo cột 'Label'
grouped = full_data.groupby('Label')

## Tạo dictionary để lưu các DataFrame tương ứng với từng nhãn
dfs = {label: group for label, group in grouped}


# Đường dẫn đến các thư mục tách ra
split_1_directory = basePath + f'/datasets/{dataset}/10 percent/train'
split_2_directory = basePath + f'/datasets/{dataset}/10 percent/val'
split_3_directory = basePath + f'/datasets/{dataset}/10 percent/test'
os.makedirs(split_1_directory, exist_ok=True)
os.makedirs(split_2_directory, exist_ok=True)
os.makedirs(split_3_directory, exist_ok=True)

for label, df_label in dfs.items():

    # chọn đặc trưng
    data_features = df_label[selected_columns]

    ''' chuẩn hóa dữ liệu Standardization '''
    data_features = np.log1p(data_features + 1)
    data_standardized = (data_features - data_features.mean()) / data_features.std()
    data_normalized = data_standardized

    # # chuyển toàn bộ df sang image
    # convert_to_img(data_normalized, label)

    # với mỗi nhãn chỉ lấy 10% dữ liệu để train model
    temp_of_ten_percent = data_normalized.sample(frac=0.1, random_state=42)  # frac=0.1 nghĩa là 10%, random_state để cố định kết quả nếu cần

    convert_to_img_percent(temp_of_ten_percent, label, split_1_directory, split_2_directory, split_3_directory)

/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:366: RuntimeWarning: divide by zero encountered in log1p
  result = func(self.values, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:366: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:366: RuntimeWarning: divide by zero encountered in log1p
  result = func(self.values, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:366: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) **